In [1]:
import torch

In [101]:
momentum = 0.1
n_features = 3
length = 10
batch_size = 32
eps = 1e-5
affine=True
track_running_stats=True
device = torch.device("cpu")
dtype = torch.float64

torch.manual_seed(0)
x1 = torch.randn((batch_size, n_features, length), dtype=dtype, requires_grad=True, device=device)
x2 = torch.randn(batch_size, n_features, length, dtype=dtype, requires_grad=True, device=device)
x1.shape

norm = torch.nn.BatchNorm1d(n_features, momentum=momentum, eps=eps, affine=affine, track_running_stats=track_running_stats, dtype=dtype)
print(norm.running_mean, norm.running_var)
out1 = norm(x1)
running_mean_torch, running_var_torch = norm.running_mean, norm.running_var
print(running_mean_torch, running_var_torch)


gamma = torch.ones_like(x1, dtype=dtype)
bias = torch.zeros_like(x1, dtype=dtype)
# running_mean = torch.zeros_like(x1, dtype=dtype)
# running_var = torch.ones_like(x1, dtype=dtype)
running_mean = torch.zeros(n_features, dtype=dtype)

mean = x1.mean(dim=0, keepdim=False)
var = x1.var(dim=0, keepdim=False, unbiased=False)

out_man = (x1 - mean) / torch.sqrt(var + eps)
out_man = gamma * out_man + bias

print(f"correct forward: {torch.allclose(out_man, out1, atol=1e-15)}")

running_mean = (1 - momentum) * running_mean + momentum * mean
running_var = (1 - momentum) * running_var + momentum * var * (batch_size / (batch_size - 1))

print(running_mean.shape, running_mean_torch.shape)

print(f"correct running mean: {torch.allclose(running_mean, running_mean_torch.view(1, n_features, 1), atol=1e-15)}")



tensor([0., 0., 0.], dtype=torch.float64) tensor([1., 1., 1.], dtype=torch.float64)
tensor([-0.0083,  0.0064,  0.0038], dtype=torch.float64) tensor([0.9984, 1.0013, 0.9975], dtype=torch.float64)
correct forward: False


RuntimeError: The size of tensor a (3) must match the size of tensor b (10) at non-singleton dimension 1

In [13]:
a = torch.randn((3,5, 6))
norm = torch.nn.BatchNorm1d(num_features=5, eps=1e-5)
norm(a) == (a - a.mean(dim=0, keepdim=True)) / torch.sqrt(a.var(dim=0, keepdim=True, unbiased=False) + 1e-5)

tensor([[[False, False, False, False, False, False],
         [False, False, False, False, False, False],
         [False, False, False, False, False, False],
         [False, False, False, False, False, False],
         [False, False, False, False, False, False]],

        [[False, False, False, False, False, False],
         [False, False, False, False, False, False],
         [False, False, False, False, False, False],
         [False, False, False, False, False, False],
         [False, False, False, False, False, False]],

        [[False, False, False, False, False, False],
         [False, False, False, False, False, False],
         [False, False, False, False, False, False],
         [False, False, False, False, False, False],
         [False, False, False, False, False, False]]])